In [1]:
import parser_funcs
import numpy as np
import os
import math
from scipy.special import kn
from scipy.interpolate import interp1d
from scipy.interpolate import SmoothBivariateSpline
from scipy.optimize import brentq
from scipy.integrate import quad
from scipy.interpolate import griddata
from scipy.optimize import minimize_scalar
from scipy.optimize import root_scalar
import sys
import glob
import random
import subprocess as subp
#import matplotlib.pyplot as plt

In [2]:
#Units
gev=1;mev=1e-3*gev;kev=1e-6*gev;

#constants
hbar=float(1.054*1e-34/(1.6e-19)/(1e9));
speed_of_light=3e8;conversion=hbar**2*speed_of_light**2;
alpha_em=1.0/137.035999074;

#masses
mp=0.938272046*gev;melec=511*kev;mmuon=105.658*mev;
mpi=134.9767*mev;mpip=139.57018*mev;mkaon=0.493667*gev;
mj_psi=3.097*gev;

In [3]:
def mom(arr):
    return math.sqrt(arr[3]**2+arr[1]**2+arr[2]**2)
    
def theta(arr):
    return math.acos(arr[3]/mom(arr))

def phi(arr):
    return math.atan(arr[0]/arr[1])

def cos_theta(arr):
    return arr[3]/mom(arr)

def theta_dif(arr1,arr2):
    return math.acos(dp(arr1[1:4],arr2[1:4])/mom(arr1)/mom(arr2))

#Expect each element of arr to be 4 elements. Expects px py pz E.
def invariant_mass(arr):
    tot = [0,0,0,0]
    for line in arr:
        for i in range(4):
            tot[i]+=line[i]
    return tot[0]**2-tot[3]**2-tot[1]**2-tot[2]**2

def cos_angle_between(arr1,arr2):
    return (arr1[3]*arr2[3]+arr1[1]*arr2[1]+arr1[2]*arr2[2])/mom(arr1)/mom(arr2)

In [4]:
def momvec(arr):
    return [arr[1],arr[2],arr[3]]
def startvec(arr):
    return [arr[5],arr[6],arr[7]]
def decaypoint(arr):
    return math.sqrt(arr[9]**2+arr[10]**2+arr[11]**2)
def dp(a1,a2):
    tot=0
    for i in range(len(a1)):
        tot+=a1[i]*a2[i]
    return tot
def veclen(a1):
    return dp(a1,a1)**0.5
def sub(a1,a2):
    return [a1[i]-a2[i] for i in range(len(a1))]
def add(a1,a2):
    return [a1[i]+a2[i] for i in range(len(a1))]
def mul(c,a1):
    return [c*a1[i] for i in range(len(a1))]
def plane_cross(p,o,r,n):
    bot=dp(n,p)
    if bot == 0:
        return -1
    a = dp(n,sub(r,o))/bot
    point = add(o,mul(a,p))
    R=sub(r,point)
    return point,dp(R,R)**0.5

In [5]:
def update_param(b,replacement_array):
    for i in range(len(b)):
        line = b[i].split()
        if len(line)<2:
            continue
        for rep in replacement_array:
            if line[0]==rep[0]:
                b[i]=line[0]+' '+str(rep[1])+'\n'
    return b
def record_list(outfile,data):
    with open(outfile,'w') as w:
        for line in data:
            tmpstr=""
            for elem in line:
                tmpstr+=str(elem)+' '
            tmpstr=tmpstr[:-1]
            tmpstr+='\n'
            w.write(tmpstr)
def append_list(outfile,data):
    with open(outfile,'a') as w:
        for line in data:
            tmpstr=""
            for elem in line:
                tmpstr+=str(elem)+' '
            tmpstr=tmpstr[:-1]
            tmpstr+='\n'
            w.write(tmpstr)

In [6]:
#l_i = dist_i/speed_of_light/lifetime*mass
def dec_prob(l1,l2):
    return math.exp(-l1)-math.exp(-l2)
def dec_loc(l1,l2):
    try:
        return -math.log(math.exp(-l1)-random.random()*(math.exp(-l1)-math.exp(-l2)))
    except:
        return l1
def solve_dec_loc(l1,l2,pos):
    return (10**pos-math.exp(-l1))/(math.exp(-l2)-math.exp(-l1))
def dec_loc_set(l1,l2,pos):
    try:
        return -math.log(math.exp(-l1)-pos*(math.exp(-l1)-math.exp(-l2)))
    except:
        return l1
#def Rescale_Events(dat,mx2,lifetime,tot_weight,eps,epset,events):
#    prob=0
#    life=lifetime*(epset/eps)**2
#    l=1.0/speed_of_light/life*mx2
#    for u in dat:
#        prob+=dec_prob(u[1]*l/u[0],u[2]*l/u[0])
#    return prob/tot_weight*events*(eps/epset)**2

In [7]:
def Event_Parser_a_gamma(event_file, cut, rescale):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]                
                break
    dat2 = [line.split() for line in dat]
    sum_line= dat2[-1]
    weight_tab=[float(line[2]) for line in dat2 if len(line)==3 and line[0]=="event"]
    DP_tab=[[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1\
                and line[0]=='Dark_Photon']
    gam_tab = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1\
                and (line[0]=='Decay_Photon')]
    ax_tab = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1\
                and (line[0]=='Decay_Axion')]
    mdp = float(sum_line[3])
    malp = float(sum_line[2])
    nevents = float(sum_line[1])*rescale
    POT = float(sum_line[10])
    gagp=float(sum_line[6])
    eps=float(sum_line[4])
    dp_lifetime=hbar/float(sum_line[8])
    eff=float(sum_line[11])
    nevents,weight_tab,DP_tab,gam_tab,ax_tab=cut(nevents,weight_tab,DP_tab,gam_tab,ax_tab)
    return [float(mdp),float(malp),gagp,eps,eff,dp_lifetime,nevents,weight_tab,DP_tab,gam_tab,ax_tab]

In [8]:
#Dummy cut for when I don't need cut2. Cut2 is used to
#check for geometry.
def Return_True(dat,DPlist,gammalist,mA,l):
    return True
#This assumes production is through kinetic mixing
def Rescale_Events_a_Gamma(dat,DPlist,gammalist,mA,ma,lifetime,tot_weight,gagpg,gagpgset,eps,epsset,events,cut,state):
    random.setstate(state)
    prob=0
    life=lifetime*(gagpgset/gagpg)**2
    l=1.0/speed_of_light/life*mA
    for i,u in enumerate(dat):
        ptmp=dec_prob(u[1]*l/u[0],u[2]*l/u[0])
        if cut(u,DPlist[i],gammalist[i],mA,l):
            prob+=ptmp
        else:
            continue
    return prob/tot_weight*events*(eps/epsset)**2
#This assumes production is through DAP
def Rescale_Events_a_Gamma_DAP(dat,DPlist,gammalist,mA,ma,lifetime,tot_weight,gagpg,gagpgset,eps,epsset,events,cut,state):
    random.setstate(state)
    prob=0
    life=lifetime*(gagpgset/gagpg)**2
    l=1.0/speed_of_light/life*mA
    for i,u in enumerate(dat):
        ptmp=dec_prob(u[1]*l/u[0],u[2]*l/u[0])
        if cut(u,DPlist[i],gammalist[i],mA,l):
            prob+=ptmp
        else:
            continue
    return prob/tot_weight*events*(gagpg/gagpgset)**2

In [108]:
def calc_params(file,evnt,rescale,cut,cut2=Return_True):
    #try:
    mdp,malp,gagpg_eval,eps_eval,eff,dp_lifetime,nevents,weight_tab,DP_tab,gam_tab,ax_tab=Event_Parser_a_gamma(file,cut,rescale)
    #except:
    #    print("Exception", file)
    #    return [-1,-1,-1]
    if nevents==0:
        return [-1,-1,-1]
    dat = [[mom(u),u[14],u[15]] for u in DP_tab]
    tot_weight=0
    for i in weight_tab:
        tot_weight+=i
    state=random.getstate()
    evnt_rate = lambda gagpg : -1.0*Rescale_Events_a_Gamma(dat,DP_tab,gam_tab,mdp,malp,dp_lifetime,tot_weight,10**gagpg,gagpg_eval,eps_eval,eps_eval,nevents,cut2,state)
    diff_from_evnt = lambda gagpg : abs(evnt_rate(gagpg)+evnt)
    diff_from_evnt_2 = lambda gagpg : evnt-Rescale_Events_a_Gamma(dat,DP_tab,gam_tab,mdp,malp,dp_lifetime,tot_weight,gagpg,gagpg_eval,eps_eval,eps_eval,nevents,cut2,state)
    opt=minimize_scalar(evnt_rate,bounds=(-6,2),method='bounded')
    if opt.fun>-evnt:
        return [mdp,-1,-1]
    
    
    opt2=minimize_scalar(diff_from_evnt,bounds=(-8,opt.x),method='bounded')
        
    x=opt.x
    xstep=0.1
    while diff_from_evnt_2(10**x)<0:
        #print(10**x,diff_from_evnt_2(10**x))
        x+=xstep
    
    #print(diff_from_evnt_2(10**(x-xstep)))
    #print(diff_from_evnt_2(10**(x)))    
    sol = root_scalar(diff_from_evnt_2, bracket=[10**(x-xstep), 10**x], method='brentq')    
    
    return [mdp,10**opt2.x,sol.root,evnt+diff_from_evnt(opt2.x),evnt+diff_from_evnt_2(sol.root)]

In [29]:
def calc_params_DAP(file,evnt,rescale,cut,cut2=Return_True):
    #try:
    mdp,malp,gagpg_eval,eps_eval,eff,dp_lifetime,nevents,weight_tab,DP_tab,gam_tab,ax_tab=Event_Parser_a_gamma(file,cut,rescale)
    #except:
    #    print("Exception", file)
    #    return [-1,-1,-1]
    if nevents==0:
        return [-1,-1,-1]
    dat = [[mom(u),u[14],u[15]] for u in DP_tab]
    tot_weight=0
    for i in weight_tab:
        tot_weight+=i
    state=random.getstate()
    evnt_rate = lambda gagpg : -1.0*Rescale_Events_a_Gamma_DAP(dat,DP_tab,gam_tab,mdp,malp,dp_lifetime,tot_weight,10**gagpg,gagpg_eval,eps_eval,eps_eval,nevents,cut2,state)
    diff_from_evnt = lambda gagpg : abs(evnt_rate(gagpg)+evnt)
    diff_from_evnt_2 = lambda gagpg : evnt-Rescale_Events_a_Gamma_DAP(dat,DP_tab,gam_tab,mdp,malp,dp_lifetime,tot_weight,gagpg,gagpg_eval,eps_eval,eps_eval,nevents,cut2,state)
    opt=minimize_scalar(evnt_rate,bounds=(-7,2),method='bounded')
    if opt.fun>-evnt:
        return [mdp,-1,-1]
    
    
    opt2=minimize_scalar(diff_from_evnt,bounds=(-8,opt.x),method='bounded')
        
    x=opt.x
    xstep=0.1
    while diff_from_evnt_2(10**x)<0:
        #print(10**x,diff_from_evnt_2(10**x))
        x+=xstep
    
    #print(diff_from_evnt_2(10**(x-xstep)))
    #print(diff_from_evnt_2(10**(x)))    
    sol = root_scalar(diff_from_evnt_2, bracket=[10**(x-xstep), 10**x], method='brentq')    
    
    return [mdp,10**opt2.x,sol.root,evnt+diff_from_evnt(opt2.x),evnt+diff_from_evnt_2(sol.root)]

In [28]:
def calc_params_list(event_file_arr,mass_tots,cut,cut2=Return_True):
    param_arr=[]
    for file in event_file_arr:
        mass=file.split('_')[-1].split('m')[0]
        for x in mass_tots:
            if x[0] == mass:
                #print(x,file)
                param_arr+=[calc_params(file,3,float(x[1]),cut,cut2)]
                break
    return param_arr

In [33]:
def calc_params_list_DAP(event_file_arr,mass_tots,cut,cut2=Return_True):
    param_arr=[]
    for file in event_file_arr:
        mass=file.split('_')[-1].split('m')[0]
        for x in mass_tots:
            if x[0] == mass:
                #print(x,file)
                param_arr+=[calc_params_DAP(file,3,float(x[1]),cut,cut2)]
                break
    return param_arr

In [19]:
def Event_Parser(event_file, cut):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]                
                break
    dat2 = [line.split() for line in dat]
    sum_line= dat2[-1]
    weight_tab=[float(line[2]) for line in dat2 if len(line)==3 and line[0]=="event"]
    dm2_tab=[[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1\
                and line[0]=='Dark_Matter_2']
    lep = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1\
                and (line[0]=='Decay_Electron' or line[0]=='Decay_Muon')]
    antilep = [[float(line[i]) for i in range(1,len(line))]+[line[0]] for line in dat2 if len(line)>1 and (line[0]=='Decay_Positron' or line[0]=='Decay_Antimuon')]
    mv = sum_line[2]
    mx1 = sum_line[3]
    mx2 = sum_line[4]
    nevents = float(sum_line[1])
    POT = float(sum_line[10])
    alpha_D=float(sum_line[6])
    eps=float(sum_line[5])
    dm2_lifetime=hbar/float(sum_line[7])
    eff=float(sum_line[11])
    nevents,weight_tab,dm2_tab,lep_tab,antilep_tab=cut(nevents,weight_tab,dm2_tab,lep,antilep)
    return [float(mv),float(mx1),float(mx2),alpha_D,eps,eff,dm2_lifetime,nevents,weight_tab,dm2_tab,lep,antilep]

In [37]:
min_energy_RENO=3*mev
#This checks that the decay product of the dark photon (a photon) has sufficient energy to be detected by RENO.
def RENO_cut(nevents,weight_tab,DP_tab,gam_tab,ax_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy = gam_tab[i][0]
        if energy > min_energy_RENO:
            pass_index.append(i)
    weight_tab_2 = [weight_tab[i] for i in pass_index]
    DP_tab = [DP_tab[i] for i in pass_index]
    gam_tab = [gam_tab[i] for i in pass_index]
    ax_tab = [ax_tab[i] for i in pass_index]
    nevents = nevents*sum(weight_tab_2)/sum(weight_tab)
    return nevents,weight_tab_2,DP_tab,gam_tab,ax_tab
#This cut checks that the produced photon enters the detector itself
RENO_plane_n=[0,0,1]
RENO_det=[0,0,302.562]
RENO_radius=1.438
#l is handled by the Rescale_Events algorithm.
def RENO_cut_2_debug(dat,dp,gamma,mdp,l):
    print("Reno_cut_2 Debugging!")
    print(dat)
    print(dp)
    print(l)
    print("Loc_Scaling")
    print(dat[1]/dat[0]*l,dat[2]/dat[0]*l)
    loc_scaling=dec_loc(dat[1]*l/dat[0],dat[2]*l/dat[0])
    print(loc_scaling)
    loc_scaling=loc_scaling/l*dat[0]
    print(loc_scaling)
    #Point where the photon decays.
    decay_point = [dp[1+i]*loc_scaling/dat[0]+dp[5+i] for i in range(3)]
    #Decay happened inside the RENO detector, no need to check photon trajectory.
    print("Decay_Point")
    print(decay_point)
    print("Photon Momentum")
    print(gamma)
    if decay_point[2] > RENO_det[2]:
        print("Decay happened in Reno detector!")
        return True
    #Momentum vector for the photon
    gmomvec=momvec(gamma);
    print(gmomvec)
    #Check that the photon is moving in the positive z direction.
    if gmomvec[2]<0:
        print("Photon is moving in the wronf direction!")
        return False
    #Calculate the point where the photon passes through the plane of the detector.
    point1g,R=plane_cross(gmomvec,decay_point,RENO_det,RENO_plane_n)
    print(point1g)
    print(R)
    #If R>RENO_radius, the gamma escaped the decay pipe before entering the RENO detector..
    if R>RENO_radius:
        print("Failed to Crossed into the RENO detector!")
        return False
    return True
def RENO_cut_2(dat,dp,gamma,mdp,l):
    #Multiply this by the dark photon momentum (dat[0]) to get the distance traveled before decay.
    loc_scaling=dec_loc(dat[1]*l/dat[0],dat[2]*l/dat[0])/l    
    #Point where the photon decays. dp[5+i] corrects for starting position.
    decay_point = [dp[1+i]*loc_scaling+dp[5+i] for i in range(3)]
    #Decay happened inside the RENO detector, no need to check photon trajectory.
    if decay_point[2] > RENO_det[2]:
        return True
    #Momentum vector for the photon
    gmomvec=momvec(gamma);
    #Check that the photon is moving in the positive z direction.
    if gmomvec[2]<0:
        return False
    #Calculate the point where the photon passes through the plane of the detector.
    point1g,R=plane_cross(gmomvec,decay_point,RENO_det,RENO_plane_n)
    #If R>RENO_radius, the gamma escaped the decay pipe before entering the RENO detector..
    if R>RENO_radius:
        return False
    return True
RENO_plane_n=[0,0,1]
alt_RENO_det=[0,0,301.1]
alt_RENO_radius=2.108
def alt_RENO_cut_2(dat,dp,gamma,mdp,l):
    #Multiply this by the dark photon momentum (dat[0]) to get the distance traveled before decay.
    loc_scaling=dec_loc(dat[1]*l/dat[0],dat[2]*l/dat[0])/l    
    #Point where the photon decays. dp[5+i] corrects for starting position.
    decay_point = [dp[1+i]*loc_scaling+dp[5+i] for i in range(3)]
    #Decay happened inside the RENO detector, no need to check photon trajectory.
    if decay_point[2] > alt_RENO_det[2]:
        return True
    #Momentum vector for the photon
    gmomvec=momvec(gamma);
    #Check that the photon is moving in the positive z direction.
    if gmomvec[2]<0:
        return False
    #Calculate the point where the photon passes through the plane of the detector.
    point1g,R=plane_cross(gmomvec,decay_point,alt_RENO_det,RENO_plane_n)
    #If R>RENO_radius, the gamma escaped the decay pipe before entering the RENO detector..
    if R>alt_RENO_radius:
        return False
    return True

min_energy_NEOS=0.3*mev
def NEOS_cut(nevents,weight_tab,DP_tab,gam_tab,ax_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy = gam_tab[i][0]
        if energy > min_energy_NEOS:
            pass_index.append(i)
    weight_tab_2 = [weight_tab[i] for i in pass_index]
    DP_tab = [DP_tab[i] for i in pass_index]
    gam_tab = [gam_tab[i] for i in pass_index]
    ax_tab = [ax_tab[i] for i in pass_index]
    nevents = nevents*sum(weight_tab_2)/sum(weight_tab)
    return nevents,weight_tab_2,DP_tab,gam_tab,ax_tab

NEOS_det=[0,0,23.2525]
NEOS_plane_n=[0,0,1]
NEOS_radius=0.5015
def NEOS_cut_2(dat,dp,gamma,mdp,l):
    #Multiply this by the dark photon momentum (dat[0]) to get the distance traveled before decay.
    loc_scaling=dec_loc(dat[1]*l/dat[0],dat[2]*l/dat[0])/l    
    #Point where the photon decays. dp[5+i] corrects for starting position.
    decay_point = [dp[1+i]*loc_scaling+dp[5+i] for i in range(3)]
    #Decay happened inside the RENO detector, no need to check photon trajectory.
    if decay_point[2] > NEOS_det[2]:
        return True
    #Momentum vector for the photon
    gmomvec=momvec(gamma);
    #Check that the photon is moving in the positive z direction.
    if gmomvec[2]<0:
        return False
    #Calculate the point where the photon passes through the plane of the detector.
    point1g,R=plane_cross(gmomvec,decay_point,NEOS_det,NEOS_plane_n)
    #If R>RENO_radius, the gamma escaped the decay pipe before entering the RENO detector..
    if R>NEOS_radius:
        return False
    return True

In [1]:
FASER1_pos=[0,0,480.75]
FASER1_n=[0,0,1]
FASER1_radius=0.1
FASER2_pos=[0,0,482.5]
FASER2_n=[0,0,1]
FASER2_radius=1
FASER_energy_cut=100
FASER_evnt=3
def FASER1_Cut(nevents,weight_tab,dp_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy = lep_tab[i][0]+antilep_tab[i][0]
        if energy>FASER_energy_cut:
            pointl,Rl=plane_cross(momvec(lep_tab[i]),startvec(lep_tab[i]),FASER1_pos,FASER1_n)
            pointa,Ra=plane_cross(momvec(antilep_tab[i]),startvec(antilep_tab[i]),FASER1_pos,FASER1_n)
            #print(Ra,Rl,pointa,pointl,veclen(sub(pointl,pointa)))
            if Ra<FASER1_radius and Rl<FASER1_radius:
                pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)
    #print(len(pass_index),len(weight_tab))
    weight_tab = [weight_tab[i] for i in pass_index]
    dp_tab = [dp_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dp_tab,lep_tab,antilep_tab

def FASER2_Cut(nevents,weight_tab,dp_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        energy = lep_tab[i][0]+antilep_tab[i][0]
        if energy>FASER_energy_cut:
            pointl,Rl=plane_cross(momvec(lep_tab[i]),startvec(lep_tab[i]),FASER2_pos,FASER2_n)
            pointa,Ra=plane_cross(momvec(antilep_tab[i]),startvec(antilep_tab[i]),FASER2_pos,FASER2_n)
            #print(Ra,Rl,pointa,pointl,veclen(sub(pointl,pointa)))
            if Ra<FASER2_radius and Rl<FASER2_radius:
                pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)
    #print(len(pass_index),len(weight_tab))
    weight_tab = [weight_tab[i] for i in pass_index]
    dp_tab = [dp_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dp_tab,lep_tab,antilep_tab

In [2]:
MATHUSLA_energy_cut=1
MATHUSLA_efficiency=1
MATHUSLA_evnt=4
MATHUSLA_det=[0,120,200]
MATHUSLA_n=[0,1,0]
def Mathusla_Cut(nevents,weight_tab,dp_tab,lep_tab,antilep_tab):
    pass_index=[]
    for i in range(len(weight_tab)):
        if lep_tab[i][0]>MATHUSLA_energy_cut and antilep_tab[i][0]>MATHUSLA_energy_cut:
            pass_index.append(i)
    nevents*=len(pass_index)/len(weight_tab)*MATHUSLA_efficiency
    weight_tab = [weight_tab[i] for i in pass_index]
    dp_tab = [dp_tab[i] for i in pass_index]
    lep_tab = [lep_tab[i] for i in pass_index]
    antilep_tab = [antilep_tab[i] for i in pass_index]
    return nevents,weight_tab,dp_tab,lep_tab,antilep_tab

In [3]:
def Mathusla_Cut_2(dat,dp,lep,alep,mdp,l):
    loc_scaling=dec_loc(dat[1]*l/dat[0],dat[2]*l/dat[0])*l/mom(dp)
    decay_point=[dp[1+i]*loc_scaling+dp[5+i] for i in range(3)]
    lmomvec=momvec(lep);amomvec=momvec(alep);
    pointl,Rl=plane_cross(lmomvec,decay_point,MATHUSLA_det,MATHUSLA_n)
    if abs(pointl[2]-200)>100 or abs(pointl[0])>100:
        return False
    pointa,Ra=plane_cross(amomvec,decay_point,MATHUSLA_det,MATHUSLA_n)
    if abs(pointa[2]-200)>100 or abs(pointa[0])>100:
        return False
    if veclen(sub(pointl,pointa))>0.01:
        return True
    return False

In [22]:
def load_mass_tot_file(file):
    with open(file) as ef:
        dat=ef.read().splitlines()
        dat=[line.split('\t') for line in dat]
    return dat

In [129]:
tst=load_mass_tot_file("data/reactor_production/reno_epsilon_1e-8.dat")

In [13]:
#Generates output files to be run by BDNMC.
def run_output_file(file,outfile,out_prepend,samplesize,data_file,eps=0):
    with open(file) as f:
        b=f.readlines()
    mdpmev=data_file.split('_')[-1].split('mev')[0]
    mdp=str(float(data_file.split('_')[-1].split('mev')[0])/1000)
    gagpg=1e-4
    if float(mdp) < 1e-4:
        gagpg = 5e-3
    rep_arr=[["epsilon",str(eps)],["gagpg",str(gagpg)],["dark_photon_mass",mdp],["particle_list_file",data_file],["samplesize",samplesize],["output_file",out_prepend+"_{}mev.dat".format(mdpmev)]]
    b=update_param(b,rep_arr)
    with open(outfile,'w') as f:
            f.writelines(b)
    subp.call(["./build/main", outfile])

In [19]:
file_arr=glob.glob("data/reactor_dp_DAP/*")

In [94]:
for file in file_arr:
    run_output_file('adp_cards/run_reno_adp.dat',"adp_cards/run_reno_auto.dat","Events_ADP_3/reno_adp",20000,file)

In [47]:
for file in file_arr:
    run_output_file('adp_cards/run_reno_adp.dat',"adp_cards/run_reno_auto.dat","Events_ADP_DAP/reno_adp",20000,file)

In [109]:
event_file_arr=glob.glob("Events_ADP_3/reno_adp*")
mass_tots=load_mass_tot_file("data/reactor_production/reno_epsilon_1e-8.dat")
param_arr=calc_params_list(event_file_arr,mass_tots,RENO_cut)
record_list("reno_1e-8.dat",param_arr)

In [23]:
mass_tots=load_mass_tot_file("data/reactor_production/reno_epsilon_0.dat")

In [57]:
mdp,malp,gagpg_eval,eps_eval,eff,dp_lifetime,nevents,weight_tab,DP_tab,gam_tab,ax_tab=Event_Parser_a_gamma("Events_ADP_DAP/reno_adp_2.mev.dat",RENO_cut,5.729711675878356e9)

In [58]:
nevents

318224.0493888325

In [40]:
event_file_arr

['Events_ADP_DAP/reno_adp_2.mev.dat']

In [53]:
mass_tots=load_mass_tot_file("data/reactor_production/reno_epsilon_0.dat")

In [54]:
mass_tots

[['0.00001', '1.8071675314396395e13'],
 ['0.00002', '1.777185386953287e13'],
 ['0.00005', '1.7247551401638375e13'],
 ['0.00007000000000000001', '1.608644241132703e13'],
 ['0.0001', '1.4853897275248088e13'],
 ['0.0002', '1.026602912587326e13'],
 ['0.0005', '2.0260057595491511e12'],
 ['0.0007', '5.897686085967267e11'],
 ['0.001', '8.203558434571228e10'],
 ['0.0014', '5.392915908546829e9'],
 ['0.0018000000000000002', '2.866581967299119e8'],
 ['0.002', '5.7297116758783564e7'],
 ['0.0022', '1.1606670966760106e7'],
 ['0.0024', '2.0725468513724452e6'],
 ['0.0025499999999999997', '525718.8109132552'],
 ['0.0026000000000000003', '323384.65711578244'],
 ['0.00265', '197725.58026672775'],
 ['0.0027', '119865.02356920853'],
 ['0.00275', '72358.37235081565'],
 ['0.0028', '40773.50143811454'],
 ['0.003', '3125.8676969400863'],
 ['0.0032', '73.12538830530062'],
 ['0.0032500000000000003', '21.16761394757204'],
 ['0.0033', '3.624716732867972'],
 ['0.00335', '0.22363112148886785'],
 ['0.0034', '0.000960

In [39]:
event_file_arr=glob.glob("Events_ADP_DAP/reno_adp*")
mass_tots=load_mass_tot_file("data/reactor_production/reno_epsilon_0.dat")
param_arr=calc_params_list_DAP(event_file_arr,mass_tots,RENO_cut)
record_list("reno_DAP.dat",param_arr)

In [37]:
event_file_arr=glob.glob("Events_ADP_3/reno_adp*")
mass_tots=load_mass_tot_file("data/reactor_production/reno_epsilon_1e-9.dat")
param_arr=calc_params_list(event_file_arr,mass_tots,RENO_cut)
record_list("reno_1e-9.dat",param_arr)

In [26]:
file_arr=glob.glob("data/reactor_dp_NEOS/*")
for file in file_arr:
    run_output_file('adp_cards/run_neos_adp.dat',"adp_cards/run_neos_auto.dat","Events_ADP_3/neos_adp",20000,file)

In [27]:
event_file_arr=glob.glob("Events_ADP_3/neos_adp*")
mass_tots=load_mass_tot_file("data/reactor_production/neos_epsilon_1e-8.dat")
param_arr=calc_params_list(event_file_arr,mass_tots,NEOS_cut)
record_list("neos_1e-8.dat",param_arr)

In [30]:
event_file_arr=glob.glob("Events_ADP_3/neos_adp*")
mass_tots=load_mass_tot_file("data/reactor_production/neos_epsilon_1e-9.dat")
param_arr=calc_params_list(event_file_arr,mass_tots,NEOS_cut)
record_list("neos_1e-9.dat",param_arr)

In [101]:
file_arr=[ 'data/reactor_dp_decaypipe/dp_0.001mev.dat']

In [103]:
file_arr=glob.glob("data/reactor_dp_decaypipe/*")
for file in file_arr:
    run_output_file('adp_cards/run_reno_decaypipe_adp.dat',"adp_cards/run_reno_decaypipe_auto.dat","Events_ADP_3/reno_decaypipe_adp",50000,file)

In [110]:
event_file_arr=glob.glob("Events_ADP_3/reno_decaypipe_adp*")
mass_tots=load_mass_tot_file("data/reactor_production/reno_decaypipe_epsilon_1e-8.dat")
param_arr=calc_params_list(event_file_arr,mass_tots,RENO_cut,cut2=RENO_cut_2)
record_list("reno_decaypipe_1e-8.dat",param_arr)

In [ ]:
event_file_arr=glob.glob("Events_ADP_3/reno_decaypipe_adp*")
mass_tots=load_mass_tot_file("data/reactor_production/reno_decaypipe_epsilon_1e-9.dat")
param_arr=calc_params_list(event_file_arr,mass_tots,RENO_cut,cut2=RENO_cut_2)
record_list("reno_decaypipe_1e-9.dat",param_arr)

In [105]:
file_arr=glob.glob("data/reactor_dp_decaypipe/*")
for file in file_arr:
    run_output_file('adp_cards/run_reno_decaypipe2_adp.dat',"adp_cards/run_reno_decaypipe2_auto.dat","Events_ADP_3/reno_decaypipe2_adp",50000,file)

In [111]:
event_file_arr=glob.glob("Events_ADP_3/reno_decaypipe2_adp*")
mass_tots=load_mass_tot_file("data/reactor_production/reno_decaypipe_epsilon_1e-8.dat")
param_arr=calc_params_list(event_file_arr,mass_tots,RENO_cut,cut2=alt_RENO_cut_2)
record_list("reno_decaypipe2_1e-8.dat",param_arr)

In [137]:
file_arr=glob.glob("data/reactor_dp_decaypipe_NEOS/*")
for file in file_arr:
    run_output_file('adp_cards/run_neos_decaypipe_adp.dat',"adp_cards/run_neos_decaypipe_auto.dat","Events_ADP_3/neos_decaypipe_adp",100000,file)

In [138]:
event_file_arr=glob.glob("Events_ADP_3/neos_decaypipe_adp*")
mass_tots=load_mass_tot_file("data/reactor_production/neos_decaypipe_epsilon_1e-8.dat")
param_arr=calc_params_list(event_file_arr,mass_tots,NEOS_cut,cut2=NEOS_cut_2)
record_list("neos_decaypipe_1e-8.dat",param_arr)